In [1]:
import cProfile
import random as rd
import time
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut

import Games.mnk as mnk
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
import Games.function_optimisation as fo
#import carcassonne as carc

In [2]:
game = ut.produce_game("mnk")
print(game.logs_data())

data in game {(0, 0): None, (0, 1): None, (0, 2): None, (1, 0): None, (1, 1): None, (1, 2): None, (2, 0): None, (2, 1): None, (2, 2): None, 'Score_p0': None, 'Score_p1': None, 'Turn': 1, 'N_available_actions': 9}
  (0, 0) (0, 1) (0, 2) (1, 0) (1, 1) (1, 2) (2, 0) (2, 1) (2, 2) Score_p0  \
0   None   None   None   None   None   None   None   None   None     None   

  Score_p1  Turn  N_available_actions  
0     None     1                    9  


In [ ]:
ut.test_mcts_tree(game_name="fo1d1p")
ut.run()

In [ ]:
base_gs = mnk.GameState(3,3,3)
base_gs.set_initial_state()
agent1 = mcts.MCTS_Player(rollouts=100, c=1, max_iterations=20, default_policy=arand.RandomPlayer())
agent2 = mcts.MCTS_Player(rollouts=20, c=1, max_iterations=100, default_policy=arand.RandomPlayer())
players = [agent1,agent2]
gs,logs = eu.play_game(base_gs, players, 5, True)
print(gs.winner)
gs.view_game_state()
print(logs)

In [4]:
cProfile.run("wins =  random_games(10000, base_gs)")

         10264365 function calls in 12.952 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.623    0.623   12.952   12.952 1794207304.py:8(random_games)
        2    0.000    0.000    0.000    0.000 1794207304.py:9(<lambda>)
        1    0.000    0.000   12.952   12.952 <string>:1(<module>)
   878085    0.971    0.000    4.956    0.000 mnk.py:105(_update_available_actions)
   878085    3.867    0.000    3.867    0.000 mnk.py:106(<listcomp>)
   888085    4.734    0.000    9.781    0.000 mnk.py:108(make_action)
    10000    0.064    0.000    0.643    0.000 mnk.py:157(duplicate)
    20000    0.332    0.000    0.332    0.000 mnk.py:161(<dictcomp>)
    10000    0.060    0.000    0.060    0.000 mnk.py:162(<listcomp>)
    10000    0.171    0.000    0.171    0.000 mnk.py:163(<dictcomp>)
    10000    0.011    0.000    0.011    0.000 mnk.py:37(__init__)
   888085    0.659    0.000    0.936    0.000 random.py:239(_randbe